In [245]:
import tweepy
from tweepy import OAuthHandler, Stream, StreamListener
from accessconfig import *
import json
import boto3
import logging
import re
import time

1. Create a class inheriting from StreamListener
2. Using that class create a Stream object
3. Connect to the Twitter API using the Stream.

In [9]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [249]:
ny_nj = [-80, 40, -72, 45, -76, 39, -74, 41]

In [18]:
def tweetParser(tweet):
    selected = {
                "created_at": tweet["created_at"],
                "text": tweet["text"],
                "user_name": tweet["user"]["name"],
                "user_description": tweet["user"]["description"],
                "place": tweet["place"]["full_name"] if tweet["place"] else None,
                "coordinates": tweet["coordinates"]["coordinates"] if tweet["coordinates"] else None,
                "hashtags": [hashtag["text"] for hashtag in tweet["entities"]["hashtags"]],
                "urls": [url["expanded_url"] for url in tweet["entities"]["urls"]],
               }

    return selected

In [246]:
class TweeterStreamListener(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        tweet =  tweetParser(all_data)
        created_at = tweet['created_at']
        text = tweet['text']
        hashtags = ''.join(tweet['hashtags'])
        place = tweet['place']
        coordinates = tweet['coordinates']
        urls = tweet['urls']
        
        find_title = matchTitle(text)
        find_skills = matchSkills(text)
        find_skills_htag = matchSkills(hashtags)
        find_city = matchLocation(text)
        find_city_htag = matchLocation(hashtags)
        find_type = matchFulltime(text)
        find_type_htag = matchFulltime(hashtags)
        
        jobpost = {
          "city": None,
          "company": None,
          "date": None,
          "job_title": None,
          "job_type": None,
          "links": None,
          "technologies": None,
          "text": None
        }
        
        jobpost['text'] = text
        jobpost['date'] = created_at
        if find_title or matchTitle(hashtags):
            print(find_title)
            jobpost["job_title"] = ' '.join(find_title)
            
            if find_skills or find_skills_htag:
                jobpost["technologies"] = ','.join(find_skills or find_skills_htag)
                
            if find_city or find_city_htag or place or coordinates:
                jobpost["city"] = ','.join(find_city or find_city_htag) or place or coordinates
            
            if find_type or find_type_htag:
                jobpost["job_type"] = ','.join(find_type or find_type_htag)
                
            if urls:
                jobpost["links"] = ', '.join(urls)
        
            print(jobpost)
            print("------------------------------------------------")
    

    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            return False


In [248]:
runtime = 180 #Tracking for 180  seconds
twitterStream = Stream(auth, TweeterStreamListener())
twitterStream.filter(languages=["en"], track=['hiring','hire','looking for', 'job'], async=True)
time.sleep(runtime)
twitterStream.disconnect()

['Software Engineer']
{'city': None, 'company': None, 'date': 'Wed Apr 17 23:36:11 +0000 2019', 'job_title': 'Software Engineer', 'job_type': None, 'links': 'http://app.jobvite.com/m?3m6nokw2', 'technologies': None, 'text': 'ServiceNow is looking for: Staff Software Engineer\nhttps://t.co/9KdXHmqswP #job'}
------------------------------------------------
['Sales Engineer']
{'city': None, 'company': None, 'date': 'Wed Apr 17 23:37:53 +0000 2019', 'job_title': 'Sales Engineer', 'job_type': None, 'links': 'https://twitter.com/i/web/status/1118659880896929793', 'technologies': None, 'text': 'JOB: Los Angeles CA USA - Pre/Post Sales Engineer Database BigData Linux Cloud Software - Database Admini: Database… https://t.co/ow7xJuTG0M'}
------------------------------------------------
['Solutions Consultant']
{'city': None, 'company': None, 'date': 'Wed Apr 17 23:38:01 +0000 2019', 'job_title': 'Solutions Consultant', 'job_type': None, 'links': 'https://twitter.com/i/web/status/1118659911041392

In [221]:
#must match
def matchTitle(txt):
    find = re.findall(r"(software|data|bi|business intelligence|sales|solutions)\s?(engineer|developer|scientist|architect|consultant)",txt,re.IGNORECASE)
    if find:
        find = [' '.join(tp) for tp in find]
        return find
    else:
        return None

In [241]:
#preferable
def matchSkills(txt):
    find = re.findall(r"python|sql|sqlalchmey|flask|pandas|etl|aws|backend|spark|streaming|jinja",txt,re.IGNORECASE)
    if find:
        return list(set(find))
    else:
        return None

In [225]:
#optional
def matchLocation(txt):
    find = re.findall(r"(new)\s?(york)|(new)\s?(jersey)|ny|nj|nyc|(jersey)\s?(city)|hoboken|brooklyn",txt,re.IGNORECASE)
    if find:
        find = [' '.join(tp) for tp in find]
        return find
    else:
        return None

In [ ]:
#optional: even if it's not in text, it's okay
def matchFulltime(txt):
    find = re.findall(r"full(\s|\-)?time",txt,re.IGNORECASE)
    if find:
        return find
    else:
        return None

## Test

In [ ]:
txt = "‼️ We’re Hiring ‼️We are currently aws for an new York.data sales engineer software engineer #python etl sql sql"

In [222]:
matchTi(txt)

['sales engineer', 'software engineer']

In [242]:
matchSkills(txt)

['aws', 'sql', 'python', 'etl']

In [160]:
a = None

In [161]:
b = ['jersey city']

In [162]:
c = 'newyork'

In [163]:
a or b or c

['jersey city']

In [167]:
','.join(a or b or c)

'aws,python'

In [169]:
x = ['www.com', 'www.joemiao.com']

In [171]:
', '.join(x)

'www.com, www.joemiao.com'

In [207]:
tp = [('sales', 'engineer'), ('software', 'engineer')]

In [210]:
ashd = [' '.join(t) for t in tp]

In [211]:
ashd

['sales engineer', 'software engineer']